In [1]:
import os

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.lib import point_flag
from pysc2.lib import stopwatch

from absl import app
from absl import flags
from future.builtins import range

import random
import math

import numpy as np
import tensorflow as tf

from pysc2.agents import base_agent
from pysc2.lib import actions
from pysc2.lib import features

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
os.environ["SC2PATH"] = "/home/nsml/StarCraftII"

In [3]:
import sys
sys.argv = ["python", "--map", "Simple64"]

# Copyright 2017 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS-IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Run an agent."""

FLAGS = flags.FLAGS

# because of Abseil's horrible design for running code underneath Colabs
# We have to pull out this ugly hack from the hat
if "flags_defined" not in globals():
    
    flags.DEFINE_bool("render", False, "Whether to render with pygame.")
    point_flag.DEFINE_point("feature_screen_size", "84",
                            "Resolution for screen feature layers.")
    point_flag.DEFINE_point("feature_minimap_size", "64",
                            "Resolution for minimap feature layers.")
    point_flag.DEFINE_point("rgb_screen_size", None,
                            "Resolution for rendered screen.")
    point_flag.DEFINE_point("rgb_minimap_size", None,
                            "Resolution for rendered minimap.")
    flags.DEFINE_enum("action_space", None, sc2_env.ActionSpace._member_names_,  # pylint: disable=protected-access
                      "Which action space to use. Needed if you take both feature "
                      "and rgb observations.")
    flags.DEFINE_bool("use_feature_units", False,
                      "Whether to include feature units.")
    flags.DEFINE_bool("disable_fog", False, "Whether to disable Fog of War.")

    flags.DEFINE_integer("max_agent_steps", 0, "Total agent steps.")
    flags.DEFINE_integer("game_steps_per_episode", None, "Game steps per episode.")
    flags.DEFINE_integer("max_episodes", 0, "Total episodes.")
    flags.DEFINE_integer("step_mul", 8, "Game steps per agent step.")

    flags.DEFINE_string("agent", "pysc2.agents.random_agent.RandomAgent",
                        "Which agent to run, as a python path to an Agent class.")
    flags.DEFINE_enum("agent_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 1's race.")

    flags.DEFINE_string("agent2", "Bot", "Second agent, either Bot or agent class.")
    flags.DEFINE_enum("agent2_race", "random", sc2_env.Race._member_names_,  # pylint: disable=protected-access
                      "Agent 2's race.")
    flags.DEFINE_enum("difficulty", "very_easy", sc2_env.Difficulty._member_names_,  # pylint: disable=protected-access
                      "If agent2 is a built-in Bot, it's strength.")

    flags.DEFINE_bool("profile", False, "Whether to turn on code profiling.")
    flags.DEFINE_bool("trace", False, "Whether to trace the code execution.")
    flags.DEFINE_integer("parallel", 1, "How many instances to run in parallel.")

    flags.DEFINE_bool("save_replay", True, "Whether to save a replay at the end.")
    
    flags.DEFINE_string("map", "CollectMineralShards", "Name of a map to use.")
    
    
flags_defined = True

In [4]:
class A3CAgent(base_agent.BaseAgent):
    
    def __init__(self):
        super(SmartAgent, self).__init__()
        
        self.qlearn = QLearningTable(actions=list(range(len(smart_actions))))
        
        self.previous_killed_unit_score = 0
        self.previous_killed_building_score = 0
        
        self.previous_action = None
        self.previous_state = None
    
    def step(self, obs):
        super(A3CAgent, self).step(obs)
        
        return actions.FUNCTIONS.no_op()

In [ ]:
class HiddenLayer:
    def __init__(self, M1, M2, f=tf.nn.tanh, use_bias=True):
        self.W = tf.Variable(tf.random_normal(shape=(M1, M2)))
        self.params = [self.W]
        self.use_bias = use_bias
        if use_bias:
            self.b = tf.Variable(np.zeros(M2).astype(np.float32))
            self.params.append(self.b)
        self.f = f

    def forward(self, X):
        if self.use_bias:
            a = tf.matmul(X, self.W) + self.b
        else:
            a = tf.matmul(X, self.W)
    return self.f(a)

In [6]:
def main(unused_argv):
    
    agent = A3CAgent()
    
    try:
        while True:
            FLAGS.map = "CollectMineralShards"
            print(FLAGS.map)
            
            with sc2_env.SC2Env(
                map_name=FLAGS.map,
                players=[sc2_env.Agent(sc2_env.Race.terran)],
                agent_interface_format=sc2_env.parse_agent_interface_format(
                  feature_screen=FLAGS.feature_screen_size,
                  feature_minimap=FLAGS.feature_minimap_size,
                  rgb_screen=FLAGS.rgb_screen_size,
                  rgb_minimap=FLAGS.rgb_minimap_size,
                  action_space=FLAGS.action_space,
                  use_feature_units=FLAGS.use_feature_units),
                step_mul=FLAGS.step_mul,
                game_steps_per_episode=FLAGS.game_steps_per_episode,
                disable_fog=FLAGS.disable_fog,
                visualize=FLAGS.render) as env:

                    agent.setup(env.observation_spec(), env.action_spec())

                    timesteps = env.reset()
                    agent.reset()

                    run_loop.run_loop([agent], env, FLAGS.max_agent_steps, FLAGS.max_episodes)

    except KeyboardInterrupt:
        pass

if __name__ == "__main__":
    app.run(main)

I1211 16:12:24.123681 140459533813568 sc_process.py:135] Launching SC2: /home/nsml/StarCraftII/Versions/Base59877/SC2_x64 -listen 127.0.0.1 -port 21494 -dataDir /home/nsml/StarCraftII/ -tempDir /tmp/sc-9rhqny6z/
I1211 16:12:24.138019 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:21494/sc2api, attempt: 0, running: True


CollectMineralShards


I1211 16:12:25.144672 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:21494/sc2api, attempt: 1, running: True
I1211 16:12:26.146149 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:21494/sc2api, attempt: 2, running: True
I1211 16:12:32.899554 140459533813568 sc2_env.py:314] Environment is ready
I1211 16:12:32.900476 140459533813568 sc2_env.py:507] Starting episode 1: [terran] on CollectMineralShards
I1211 16:12:32.938467 140459533813568 sc2_env.py:507] Starting episode 2: [terran] on CollectMineralShards
I1211 16:12:34.441949 140459533813568 sc2_env.py:725] Episode 2 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I1211 16:12:34.471040 140459533813568 sc2_env.py:507] Starting episode 3: [terran] on CollectMineralShards
I1211 16:12:35.883983 140459533813568 sc2_env.py:725] Episode 3 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I1211 16:12:35.915439 140459533813568 sc2_env.py:507] Starting episo

Took 6.573 seconds for 1076 steps: 163.694 fps
CollectMineralShards


I1211 16:12:40.540050 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:22615/sc2api, attempt: 1, running: True
I1211 16:12:41.543122 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:22615/sc2api, attempt: 2, running: True
I1211 16:12:48.407186 140459533813568 sc2_env.py:314] Environment is ready
I1211 16:12:48.408304 140459533813568 sc2_env.py:752] Environment Close
I1211 16:12:48.409437 140459533813568 sc2_env.py:507] Starting episode 1: [terran] on CollectMineralShards
I1211 16:12:48.447991 140459533813568 sc2_env.py:507] Starting episode 2: [terran] on CollectMineralShards
I1211 16:12:49.918259 140459533813568 sc2_env.py:725] Episode 2 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I1211 16:12:49.949822 140459533813568 sc2_env.py:507] Starting episode 3: [terran] on CollectMineralShards
I1211 16:12:51.421367 140459533813568 sc2_env.py:725] Episode 3 finished after 1920 game steps. Outcome: [1], reward: [0], score: 

...

Took 43.867 seconds for 7515 steps: 171.314 fps
CollectMineralShards


I1211 16:13:33.345282 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:20273/sc2api, attempt: 1, running: True
I1211 16:13:34.355322 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:20273/sc2api, attempt: 2, running: True
I1211 16:13:35.371760 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:20273/sc2api, attempt: 3, running: True
I1211 16:13:42.050522 140459533813568 sc2_env.py:314] Environment is ready
I1211 16:13:42.051589 140459533813568 sc2_env.py:752] Environment Close
I1211 16:13:42.053149 140459533813568 sc2_env.py:507] Starting episode 1: [terran] on CollectMineralShards
I1211 16:13:42.088284 140459533813568 sc2_env.py:507] Starting episode 2: [terran] on CollectMineralShards
I1211 16:13:43.600623 140459533813568 sc2_env.py:725] Episode 2 finished after 1920 game steps. Outcome: [1], reward: [0], score: [0]
I1211 16:13:43.632581 140459533813568 sc2_env.py:507] Starting episode 3: [terran] on CollectMineralSha

Took 5.790 seconds for 908 steps: 156.829 fps
CollectMineralShards


I1211 16:13:48.907790 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:16127/sc2api, attempt: 1, running: True
I1211 16:13:50.008306 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:16127/sc2api, attempt: 2, running: True
I1211 16:13:51.017104 140459533813568 remote_controller.py:167] Connecting to: ws://127.0.0.1:16127/sc2api, attempt: 3, running: True


SystemExit: 